This notebook used resting-state time-series to calculate the subject-level connectivity matrix of the Cognitive Control network as identified in [@dosenbach2007]. The connectivity matrix, of shape $N_{\text{subjects}} \times N_{\text{regions}} \times N_{\text{regions}}$, will be stored as an annotated NetCDF4 file in the following file: `data/julia2018_resting/connectivities.nc`.

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from nilearn.connectome import ConnectivityMeasure

In [3]:
DATASET = xr.open_dataset('data/julia2018_resting/timeseries.nc')

In [4]:

def fit_connectivity(timeseries: np.ndarray, kind='correlation'):
  """Extract a connectivity matrix of a timeseries array.
  
  Receives a timeseries numpy array and extracts the subject-level connectivity matrix.
  
  Args:
    timeseries: numpy array of shape (n_subjects, n_regions, n_timepoints)
    kind: string, kind of connectivity measure to extract. Defaults to 'correlation'. See
      nilearn.connectome.ConnectivityMeasure for more details.
  
  Returns:
    connectivity: numpy array of shape (n_subjects, n_regions, n_regions)
  """
  print()
  
  cm = ConnectivityMeasure(kind=kind)
  
  y = []
  # transpose {feature <-> sample} and convert to a list of subjects
  for sub_ts in timeseries:
    sub_X = sub_ts.T
    if not np.all(np.isnan(sub_X)):
      sub_y = cm.fit_transform([sub_X])[0]
    else:
      sub_y = np.empty((sub_X.shape[1], sub_X.shape[1]))
      sub_y[:] = np.nan
      
    y.append(sub_y)
 
  return np.array(y)

# fit connectivity model
connectivities = fit_connectivity(DATASET['timeseries'].values)

# prepare and store the connectivity data
DATASET['connectivity'] = xr.DataArray(connectivities, dims=['subject', 'region', 'region'])
DATASET.attrs['description'] = 'Connectivity matrices extracted from resting timeseries.'

In [5]:
# now store the dataset

DATASET.to_netcdf('data/julia2018_resting/connectivities.nc', engine='netcdf4')

In [6]:
DATASET

<xarray.Dataset>
Dimensions:                      (subject: 34, region: 39, timestep: 125,
                                  mni152_coord: 3)
Coordinates:
  * subject                      (subject) object 'AVGP01' ... 'NVGP19NEW'
  * region                       (region) object 'R IPS' 'L IPS' ... 'vmPFC'
  * mni152_coord                 (mni152_coord) object 'x' 'y' 'z'
  * timestep                     (timestep) int64 0 1 2 3 4 ... 121 122 123 124
Data variables:
    timeseries                   (subject, region, timestep) float64 -0.04544...
    regions                      (region, mni152_coord) int64 ...
    groups                       (subject) object ...
    age                          (subject) float64 ...
    sex                          (subject) object ...
    group                        (subject) object ...
    preprocessed_beh             (subject) bool ...
    preprocessed_rsfmri          (subject) bool ...
    preprocessed_tfmri           (subject) bool ...
    inverse_efficiency_score_ms  (subject) float64 ...
    false_alarm_pct              (subject) float64 ...
    connectivity                 (subject, region, region) float64 1.0 ... 1.0
Attributes:
    description:  Connectivity matrices extracted from resting timeseries.